<a href="https://colab.research.google.com/github/lcylmhlcy/Awesome-algorithm-interview/blob/master/pytorch1_0__ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! nvidia-smi

Thu Apr 25 12:15:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import collections
import os
import shutil
import tqdm

import numpy as np
import PIL.Image
import torch
import torchvision

In [0]:
print(torch.__version__)               # PyTorch version
print(torch.version.cuda)              # Corresponding CUDA version
print(torch.backends.cudnn.version())  # Corresponding cuDNN version
print(torch.cuda.get_device_name(0))   # GPU type

1.0.1.post2
10.0.130
7402
Tesla T4


In [0]:
# 判断是否有cuda支持
print(torch.cuda.is_available())

True


In [0]:
# 设置为 cuDNN benchmark 模式

# Benchmark 模式会提升计算速度，但是由于计算中有随机性，每次网络前馈结果略有差异。
torch.backends.cudnn.benchmark = True

# 如果想要避免这种结果波动，设置
torch.backends.cudnn.deterministic = True

# Note:
# 总的来说，大部分情况下，设置这个 benchmark 可以让内置的 cuDNN 的 auto-tuner 自动寻找最适合当前配置的高效算法，来达到优化运行效率的问题。
# 一般来讲，应该遵循以下准则：
# (1)如果网络的输入数据维度或类型上变化不大，设置  torch.backends.cudnn.benchmark = true  可以增加运行效率；
# (2)如果网络的输入数据在每次 iteration 都变化的话，会导致 cnDNN 每次都会去寻找一遍最优配置，这样反而会降低运行效率。

In [0]:
# 清除 GPU 存储

# 有时 Control-C 中止运行后 GPU 存储没有及时释放，需要手动清空。在 PyTorch 内部可以
torch.cuda.empty_cache()

# 或在命令行可以先使用 ps 找到程序的 PID，再使用 kill 结束该进程
ps aux | grep pythonkill -9 [pid]

# 或者直接重置没有被清空的 GPU
nvidia-smi --gpu-reset -i [gpu_id]

In [0]:
# 在训练开始时，参数的初始化是随机的，为了让每次的结果一致，我们需要设置随机种子。

# 固定GPU随机化种子
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# 为CPU设置随机种子
torch.manual_seed(args.seed)

In [0]:
import pynvml
pynvml.nvmlInit()
import os

# 查看当前显卡0，1的显存情况
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print('GPU 0:', (meminfo.used) / 1024**3, 'G')

handle = pynvml.nvmlDeviceGetHandleByIndex(1)
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print('GPU 1:', (meminfo.used) / 1024**3, 'G')

# 打印目前显存
print(os.system('nvidia-smi'))

In [0]:
# Tensor--------------------------------------------------------------------------------

In [0]:
a = torch.Tensor([[1,2,3],[4,5,6]])
print(a.type())
print(a.size())
print(a.dim())

torch.FloatTensor
torch.Size([2, 3])
2


In [0]:
# Set default tensor type. Float in PyTorch is much faster than double.
torch.set_default_tensor_type(torch.FloatTensor)

In [0]:
# Type convertions.
a = a.cuda() # convert Tensor a on CPU to Tensor a on GPU
a = a.cpu() # convert Tensor a on GPU to Tensor a on CPU

In [0]:
b = a.float() # convert a to FloatTensor
print(b.type())
c = a.long() # convert a to LongTensor
print(c.type())

torch.FloatTensor
torch.LongTensor


In [0]:
# torch.Tensor -> np.ndarray.
a = a.cuda()
a = a.cpu().numpy()  # convert to numpy on CPU

# np.ndarray -> torch.Tensor.
a1 = np.array([[1,2,3],[4,5,6]], dtype=np.int64)
b1 = torch.from_numpy(ndarray).float()
c1 = torch.from_numpy(ndarray.copy()).float()  # If ndarray has negative stride

In [0]:
# torch.Tensor 与 PIL.Image 转换
# PyTorch 中的张量默认采用 N×D×H×W 的顺序，并且数据范围在 [0, 1]，需要进行转置和规范化。

# torch.Tensor -> PIL.Image.
image = PIL.Image.fromarray(torch.clamp(tensor * 255, min=0, max=255).byte().permute(1, 2, 0).cpu().numpy())
image = torchvision.transforms.functional.to_pil_image(tensor)  # Equivalently way

# PIL.Image -> torch.Tensor.
tensor = torch.from_numpy(np.asarray(PIL.Image.open(path))).permute(2, 0, 1).float() / 255
tensor = torchvision.transforms.functional.to_tensor(PIL.Image.open(path))  # Equivalently way

In [0]:
# np.ndarray 与 PIL.Image 转换

# np.ndarray -> PIL.Image.
image = PIL.Image.fromarray(ndarray.astypde(np.uint8))

# PIL.Image -> np.ndarray.
ndarray = np.asarray(PIL.Image.open(path))

In [0]:
# 从只包含一个元素的张量中提取值
# 这在训练时统计 loss 的变化过程中特别有用。否则这将累积计算图，使 GPU 存储占用量越来越大。

value = tensor.item()

In [0]:
# 张量形变
# 张量形变常常需要用于将卷积层特征输入全连接层的情形。相比 torch.view，torch.reshape 可以自动处理输入张量不连续的情况。

tensor = torch.reshape(tensor, shape)

In [0]:
 # 打乱顺序

tensor = tensor[torch.randperm(tensor.size(0))]  # Shuffle the first dimension

In [0]:
# 水平翻转
# PyTorch 不支持 tensor[::-1] 这样的负步长操作，水平翻转可以用张量索引实现。

# Assume tensor has shape N*D*H*W.
tensor = tensor[:, :, :, torch.arange(tensor.size(3) - 1, -1, -1).long()]

In [0]:
# 复制张量
# 有三种复制的方式，对应不同的需求。

# Operation                 |  New/Shared memory | Still in computation graph |
tensor.clone()            # |        New         |          Yes               |
tensor.detach()           # |      Shared        |          No                |
tensor.detach.clone()()   # |        New         |          No                |

In [0]:
# 拼接张量
# 注意 torch.cat 和 torch.stack 的区别在于 torch.cat 沿着给定的维度拼接，而 torch.stack 会新增一维。
# 例如当参数是 3 个 10×5 的张量，torch.cat 的结果是 30×5 的张量，而 torch.stack 的结果是 3×10×5 的张量。

tensor = torch.cat(list_of_tensors, dim=0)
tensor = torch.stack(list_of_tensors, dim=0)

In [0]:
# 将整数标记转换成独热（one-hot）编码
# PyTorch 中的标记默认从 0 开始。

N = tensor.size(0)
one_hot = torch.zeros(N, num_classes).long()
one_hot.scatter_(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(N, num_classes).long())

In [0]:
# 得到非零/零元素

torch.nonzero(tensor)               # Index of non-zero elements
torch.nonzero(tensor == 0)          # Index of zero elements
torch.nonzero(tensor).size(0)       # Number of non-zero elements
torch.nonzero(tensor == 0).size(0)  # Number of zero elements

In [0]:
# 张量扩展

# Expand tensor of shape 64*512 to shape 64*512*7*7.
torch.reshape(tensor, (64, 512, 1, 1)).expand(64, 512, 7, 7)

In [0]:
# 矩阵乘法

# Matrix multiplication: (m*n) * (n*p) -> (m*p).
result = torch.mm(tensor1, tensor2)

# Batch matrix multiplication: (b*m*n) * (b*n*p) -> (b*m*p).
result = torch.bmm(tensor1, tensor2)

# Element-wise multiplication.
result = tensor1 * tensor2

In [0]:
# 计算两组数据之间的两两欧式距离

# X1 is of shape m*d.
X1 = torch.unsqueeze(X1, dim=1).expand(m, n, d)
# X2 is of shape n*d.
X2 = torch.unsqueeze(X2, dim=0).expand(m, n, d)
# dist is of shape m*n, where dist[i][j] = sqrt(|X1[i, :] - X[j, :]|^2)
dist = torch.sqrt(torch.sum((X1 - X2) ** 2, dim=2))

In [0]:
# ----模型定义------------------------------------------------------------------------------------------------

In [0]:
# 卷积层

conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=True)

# 如果卷积层配置比较复杂，不方便计算输出大小时，可以利用如下可视化工具辅助
# 链接：https://ezyang.github.io/convolution-visualizer/index.html

In [0]:
# GAP（Global average pooling）层

gap = torch.nn.AdaptiveAvgPool2d(output_size=1)

In [0]:
# 双线性汇合（bilinear pooling）

X = torch.reshape(N, D, H * W)                        # Assume X has shape N*D*H*W
X = torch.bmm(X, torch.transpose(X, 1, 2)) / (H * W)  # Bilinear pooling
assert X.size() == (N, D, D)
X = torch.reshape(X, (N, D * D))
X = torch.sign(X) * torch.sqrt(torch.abs(X) + 1e-5)   # Signed-sqrt normalization
X = torch.nn.functional.normalize(X)                  # L2 normalization

In [0]:
# 多卡同步 BN（Batch normalization）

# 当使用 torch.nn.DataParallel 将代码运行在多张 GPU 卡上时，PyTorch 的 BN 层默认操作是各卡上数据独立地计算均值和标准差，
# 同步 BN 使用所有卡上的数据一起计算 BN 层的均值和标准差，缓解了当批量大小（batch size）比较小时对均值和标准差估计不准的情况，
# 是在目标检测等任务中一个有效的提升性能的技巧。

# 链接：https://github.com/vacancy/Synchronized-BatchNorm-PyTorch

In [0]:
# 类似 BN 滑动平均
# 如果要实现类似 BN 滑动平均的操作，在 forward 函数中要使用原地（inplace）操作给滑动平均赋值。

class BN(torch.nn.Module)
    def __init__(self):
        ...
        self.register_buffer('running_mean', torch.zeros(num_features))

    def forward(self, X):
        ...
        self.running_mean += momentum * (current - self.running_mean)

In [0]:
# 计算模型整体参数量

num_parameters = sum(torch.numel(parameter) for parameter in model.parameters())

In [0]:
# 类似 Keras 的 model.summary() 输出模型信息

# 链接：https://github.com/sksq96/pytorch-summary

In [0]:
# 模型权值初始化

# 注意 model.modules() 和 model.children() 的区别：model.modules() 会迭代地遍历模型的所有子层，
# 而 model.children() 只会遍历模型下的一层。

# Common practise for initialization.
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(layer.weight, mode='fan_out',
                                      nonlinearity='relu')
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias, val=0.0)
    elif isinstance(layer, torch.nn.BatchNorm2d):
        torch.nn.init.constant_(layer.weight, val=1.0)
        torch.nn.init.constant_(layer.bias, val=0.0)
    elif isinstance(layer, torch.nn.Linear):
        torch.nn.init.xavier_normal_(layer.weight)
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias, val=0.0)

# Initialization with given tensor.
layer.weight = torch.nn.Parameter(tensor)

In [0]:
# 部分层使用预训练模型

# 注意如果保存的模型是 torch.nn.DataParallel，则当前的模型也需要是

model.load_state_dict(torch.load('model,pth'), strict=False)


# 读取整个网络
model = torch.load(PATH)
# 读取网络中的参数
model.load_state_dict(torch.load(PATH))
# 读取网络中的部分参数（本质其实就是更新字典）
pretrained_dict = torch.load(pretrained_model_weight)
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)

In [0]:
# 保存整个网络
torch.save(model, PATH) 
# 保存网络中的参数, 速度快，占空间少
torch.save(model.state_dict(),PATH)
# 选择保存网络中的一部分参数或者额外保存其余的参数
torch.save({'state_dict': model.state_dict(), 'fc_dict':model.fc.state_dict(),
            'optimizer': optimizer.state_dict(),'alpha': loss.alpha, 'gamma': loss.gamma},
             PATH)

In [0]:
# 将在 GPU 保存的模型加载到 CPU

model.load_state_dict(torch.load('model,pth', map_location='cpu'))

In [0]:
# 加载完 Pre-Trained Model 后，我们需要对其进行 Finetune。但是在此之前，我们往往需要冻结一部分的模型参数：
# 第一种方式
for p in freeze.parameters(): # 将需要冻结的参数的 requires_grad 设置为 False
    p.requires_grad = False
for p in no_freeze.parameters(): # 将fine-tuning 的参数的 requires_grad 设置为 True
    p.requires_grad = True
# 将需要 fine-tuning 的参数放入optimizer 中
optimizer.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

# 第二种方式
optim_param = []
for p in freeze.parameters(): # 将需要冻结的参数的 requires_grad 设置为 False
    p.requires_grad = False
for p in no_freeze.parameters(): # 将fine-tuning 的参数的 requires_grad 设置为 True
    p.requires_grad = True
    optim_param.append(p)
optimizer.SGD(optim_param, lr=1e-3) # 将需要 fine-tuning 的参数放入optimizer 中

In [0]:
# ----数据准备、特征提取与微调-----------------------------------------------------------------------

In [0]:
# 得到视频数据基本信息

import cv2
video = cv2.VideoCapture(mp4_path)
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video.get(cv2.CAP_PROP_FPS))
video.release()

In [0]:
# TSN 每段（segment）采样一帧视频

K = self._num_segments
if is_train:
    if num_frames > K:
        # Random index for each segment.
        frame_indices = torch.randint(
            high=num_frames // K, size=(K,), dtype=torch.long)
        frame_indices += num_frames // K * torch.arange(K)
    else:
        frame_indices = torch.randint(
            high=num_frames, size=(K - num_frames,), dtype=torch.long)
        frame_indices = torch.sort(torch.cat((
            torch.arange(num_frames), frame_indices)))[0]
else:
    if num_frames > K:
        # Middle index for each segment.
        frame_indices = num_frames / K // 2
        frame_indices += num_frames // K * torch.arange(K)
    else:
        frame_indices = torch.sort(torch.cat((                              
            torch.arange(num_frames), torch.arange(K - num_frames))))[0]
assert frame_indices.size() == (K,)
return [frame_indices[i] for i in range(K)]

In [0]:
# 提取 ImageNet 预训练模型某层的卷积特征

# VGG-16 relu5-3 feature.
model = torchvision.models.vgg16(pretrained=True).features[:-1]
# VGG-16 pool5 feature.
model = torchvision.models.vgg16(pretrained=True).features
# VGG-16 fc7 feature.
model = torchvision.models.vgg16(pretrained=True)
model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])
# ResNet GAP feature.
model = torchvision.models.resnet18(pretrained=True)
model = torch.nn.Sequential(collections.OrderedDict(
    list(model.named_children())[:-1]))

with torch.no_grad():
    model.eval()
    conv_representation = model(image)

In [0]:
# 提取 ImageNet 预训练模型多层的卷积特征

class FeatureExtractor(torch.nn.Module):
    """Helper class to extract several convolution features from the given
    pre-trained model.

    Attributes:
        _model, torch.nn.Module.
        _layers_to_extract, list<str> or set<str>

    Example:
        >>> model = torchvision.models.resnet152(pretrained=True)
        >>> model = torch.nn.Sequential(collections.OrderedDict(
                list(model.named_children())[:-1]))
        >>> conv_representation = FeatureExtractor(
                pretrained_model=model,
                layers_to_extract={'layer1', 'layer2', 'layer3', 'layer4'})(image)
    """
    def __init__(self, pretrained_model, layers_to_extract):
        torch.nn.Module.__init__(self)
        self._model = pretrained_model
        self._model.eval()
        self._layers_to_extract = set(layers_to_extract)

    def forward(self, x):
        with torch.no_grad():
            conv_representation = []
            for name, layer in self._model.named_children():
                x = layer(x)
                if name in self._layers_to_extract:
                    conv_representation.append(x)
            return conv_representation

In [0]:
# 其他预训练模型

# 链接：https://github.com/Cadene/pretrained-models.pytorch

In [0]:
# 微调全连接层

model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(512, 100)  # Replace the last fc layer
optimizer = torch.optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)

In [0]:
# 以较大学习率微调全连接层，较小学习率微调卷积层

model = torchvision.models.resnet18(pretrained=True)
finetuned_parameters = list(map(id, model.fc.parameters()))
conv_parameters = (p for p in model.parameters() if id(p) not in finetuned_parameters)
parameters = [{'params': conv_parameters, 'lr': 1e-3}, 
              {'params': model.fc.parameters()}]
optimizer = torch.optim.SGD(parameters, lr=1e-2, momentum=0.9, weight_decay=1e-4)

In [0]:
# -------模型训练------------------------------------------------------------------------------------------

In [0]:
# 常用训练和验证数据预处理

# 其中 ToTensor 操作会将 PIL.Image 或形状为 H×W×D，数值范围为 [0, 255] 的 np.ndarray 转换为形状为 D×H×W，
# 数值范围为 [0.0, 1.0] 的 torch.Tensor。

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=224,
                                             scale=(0.08, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                     std=(0.229, 0.224, 0.225)),
 ])
 val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                     std=(0.229, 0.224, 0.225)),
])

In [0]:
# 训练基本代码框架

for t in epoch(80):
    for images, labels in tqdm.tqdm(train_loader, desc='Epoch %3d' % (t + 1)):
        images, labels = images.cuda(), labels.cuda()
        scores = model(images)
        loss = loss_function(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [0]:
# 标记平滑（label smoothing）

for images, labels in train_loader:
    images, labels = images.cuda(), labels.cuda()
    N = labels.size(0)
    # C is the number of classes.
    smoothed_labels = torch.full(size=(N, C), fill_value=0.1 / (C - 1)).cuda()
    smoothed_labels.scatter_(dim=1, index=torch.unsqueeze(labels, dim=1), value=0.9)

    score = model(images)
    log_prob = torch.nn.functional.log_softmax(score, dim=1)
    loss = -torch.sum(log_prob * smoothed_labels) / N
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [0]:
# Mixup

beta_distribution = torch.distributions.beta.Beta(alpha, alpha)
for images, labels in train_loader:
    images, labels = images.cuda(), labels.cuda()

    # Mixup images.
    lambda_ = beta_distribution.sample([]).item()
    index = torch.randperm(images.size(0)).cuda()
    mixed_images = lambda_ * images + (1 - lambda_) * images[index, :]

    # Mixup loss.    
    scores = model(mixed_images)
    loss = (lambda_ * loss_function(scores, labels) 
            + (1 - lambda_) * loss_function(scores, labels[index]))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [0]:
# L1 正则化

l1_regularization = torch.nn.L1Loss(reduction='sum')
loss = ...  # Standard cross-entropy loss
for param in model.parameters():
    loss += torch.sum(torch.abs(param))
loss.backward()

In [0]:
# 不对偏置项进行 L2 正则化/权值衰减（weight decay）

bias_list = (param for name, param in model.named_parameters() if name[-4:] == 'bias')
others_list = (param for name, param in model.named_parameters() if name[-4:] != 'bias')
parameters = [{'parameters': bias_list, 'weight_decay': 0},                
              {'parameters': others_list}]
optimizer = torch.optim.SGD(parameters, lr=1e-2, momentum=0.9, weight_decay=1e-4)

In [0]:
# 梯度裁剪（gradient clipping）

torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=20)

In [0]:
# 计算 Softmax 输出的准确率

score = model(images)
prediction = torch.argmax(score, dim=1)
num_correct = torch.sum(prediction == labels).item()
accuruacy = num_correct / labels.size(0)

In [0]:
# 可视化模型前馈的计算图

# 链接：https://github.com/szagoruyko/pytorchviz

In [0]:
# 得到当前学习率

# If there is one global learning rate (which is the common case).
lr = next(iter(optimizer.param_groups))['lr']

# If there are multiple learning rates for different layers.
all_lr = []
for param_group in optimizer.param_groups:
    all_lr.append(param_group['lr'])

In [0]:
# 学习率衰减

# Reduce learning rate when validation accuarcy plateau.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True)
for t in range(0, 80):
    train(...); val(...)
    scheduler.step(val_acc)

# Cosine annealing learning rate.
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80)
# Reduce learning rate by 10 at given epochs.
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 70], gamma=0.1)
for t in range(0, 80):
    scheduler.step()    
    train(...); val(...)

# Learning rate warmup by 10 epochs.
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda t: t / 10)
for t in range(0, 10):
    scheduler.step()
    train(...); val(...)

In [0]:
# 保存与加载断点

# 注意为了能够恢复训练，我们需要同时保存模型和优化器的状态，以及当前的训练轮数。

# Save checkpoint.
is_best = current_acc > best_acc
best_acc = max(best_acc, current_acc)
checkpoint = {
    'best_acc': best_acc,    
    'epoch': t + 1,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}
model_path = os.path.join('model', 'checkpoint.pth.tar')
torch.save(checkpoint, model_path)
if is_best:
    shutil.copy('checkpoint.pth.tar', model_path)

# Load checkpoint.
if resume:
    model_path = os.path.join('model', 'checkpoint.pth.tar')
    assert os.path.isfile(model_path)
    checkpoint = torch.load(model_path)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('Load checkpoint at epoch %d.' % start_epoch)

In [0]:
# 计算准确率、查准率（precision）、查全率（recall）

# data['label'] and data['prediction'] are groundtruth label and prediction 
# for each image, respectively.
accuracy = np.mean(data['label'] == data['prediction']) * 100

# Compute recision and recall for each class.
for c in range(len(num_classes)):
    tp = np.dot((data['label'] == c).astype(int),
                (data['prediction'] == c).astype(int))
    tp_fp = np.sum(data['prediction'] == c)
    tp_fn = np.sum(data['label'] == c)
    precision = tp / tp_fp * 100
    recall = tp / tp_fn * 100

In [0]:
# -------其它------------------------------------------------------------------------------------------

In [0]:
# 模型定义

# 建议有参数的层和汇合（pooling）层使用 torch.nn 模块定义，激活函数直接使用 torch.nn.functional。
# torch.nn 模块和 torch.nn.functional 的区别在于，torch.nn 模块在计算时底层调用了 torch.nn.functional，
# 但 torch.nn 模块包括该层参数，还可以应对训练和测试两种网络状态。使用 torch.nn.functional 时要注意网络状态，如

def forward(self, x):
    ...
    x = torch.nn.functional.dropout(x, p=0.5, training=self.training)

# model(x) 前用 model.train() 和 model.eval() 切换网络状态。

# 不需要计算梯度的代码块用 with torch.no_grad() 包含起来。model.eval() 和 torch.no_grad() 的区别在于，
# model.eval() 是将网络切换为测试状态，例如 BN 和随机失活（dropout）在训练和测试阶段使用不同的计算方法。
# torch.no_grad() 是关闭 PyTorch 张量的自动求导机制，以减少存储使用和加速计算，得到的结果无法进行 loss.backward()。

# torch.nn.CrossEntropyLoss 的输入不需要经过 Softmax。torch.nn.CrossEntropyLoss 等价于 torch.nn.functional.log_softmax + torch.nn.NLLLoss。
# loss.backward() 前用 optimizer.zero_grad() 清除累积梯度。optimizer.zero_grad() 和 model.zero_grad() 效果一样。

In [0]:
# PyTorch 性能与调试

# torch.utils.data.DataLoader 中尽量设置 pin_memory=True，对特别小的数据集如 MNIST 设置 pin_memory=False 反而更快一些。
# num_workers 的设置需要在实验中找到最快的取值。

# 用 del 及时删除不用的中间变量，节约 GPU 存储。

# 使用 inplace 操作可节约 GPU 存储，如

x = torch.nn.functional.relu(x, inplace=True)


# 减少 CPU 和 GPU 之间的数据传输。例如如果你想知道一个 epoch 中每个 mini-batch 的 loss 和准确率，
# 先将它们累积在 GPU 中等一个 epoch 结束之后一起传输回 CPU 会比每个 mini-batch 都进行一次 GPU 到 CPU 的传输更快。

# 使用半精度浮点数 half() 会有一定的速度提升，具体效率依赖于 GPU 型号。需要小心数值精度过低带来的稳定性问题。

# 时常使用 assert tensor.size() == (N, D, H, W) 作为调试手段，确保张量维度和你设想中一致。

# 除了标记 y 外，尽量少使用一维张量，使用 n*1 的二维张量代替，可以避免一些意想不到的一维张量计算结果。

# 统计代码各部分耗时

with torch.autograd.profiler.profile(enabled=True, use_cuda=False) as profile:
    ...
print(profile)

# 或者在命令行运行

python -m torch.utils.bottleneck main.py

In [0]:
# PyTorch中通过Dataloader加载图片，使用十分方便。但当加载图片较多并且需要做较多变换时，加载的速度很慢，
# 会出现加载数据过慢（即使已经使用了多个worker），GPU空闲等待数据加载的情况。
# ------------------------------------------------------------------------------------------------------------------
# 1. 读取jpg图片加速
#加载数据慢的一个很重要的原因的读取图片本身就比较慢，尤其是jpg图片解码更是耗时。
# 如果你是读取jpg图片的话建议使用jpeg4py进行读取，只需要替换一行代码就搞定了。

#具体的使用例子：
import jpeg4py as jpeg
import cv2

if __name__ == "__main__":
    img_path = '1.jpg'
    img = jpeg.JPEG(img_path).decode()
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
# 解码出来的直接就是numpy数组，可以直接用opencv显示并进行相应的操作，但需要注意的是通道顺序是RGB，opencv默认的BGR。
# 自己在小数据上测试了一下，使用opencv读取一个epoch需要7.5s，使用jpeg4py只需要3~4s，提速了近一倍。
# https://github.com/ajkxyz/jpeg4py
# ------------------------------------------------------------------------------------------------------------------
# 2. 预取下一次迭代需要的数据
# 提前预读数据，也是看了这篇文章得到的方法。
# https://zhuanlan.zhihu.com/p/66145913

# 一个使用的例子：

class data_prefetcher():
    def __init__(self, loader):
        self.loader = iter(loader)
        # self.stream = torch.cuda.Stream()
        self.preload()

    def preload(self):
        try:
            self.next_data = next(self.loader)
        except StopIteration:
            self.next_input = None
            return
        # with torch.cuda.stream(self.stream):
        #     self.next_data = self.next_data.cuda(non_blocking=True)
            
    def next(self):
        # torch.cuda.current_stream().wait_stream(self.stream)
        data = self.next_data
        self.preload()
        return data

###正常定义train_loader###

prefetcher = data_prefetcher(train_loader)
data = prefetcher.next()
i = 0
while data is not None:
    print(i, len(data))
    i += 1
    data = prefetcher.next()
    
# 由于本地环境没有GPU，使用没有使用cuda，简单的使用预加载，速度可以从3.5s提高到2.25s。

In [0]:
# 利用 torch.nn.DataParallel 进行多 GPU 训练

import torch
import torch.nn as nn
import torchvision.models as models
# 生成模型
# 利用 torch.nn.DataParallel 进行载入模型，默认使用所有GPU（可以用 CUDA_VISIBLE_DEVICES 设置所使用的 GPU）
model = nn.DataParallel(models.resnet18())

# 冻结参数
for param in model.module.layer4.parameters():
    param.requires_grad = False
param_optim = filter(lambda p:p.requires_grad, model.parameters())

# 设置测试模式
model.module.layer4.eval()

# 保存模型参数（读取所保存模型参数后，再进行并行化操作，否则无法利用之前的代码进行读取）
torch.save(model.module.state_dict(),'./CheckPoint.pkl')